In [1]:
#nanti amend SAP no bina puri & OM material 500724 yg BG

#Manually check column closing yg mne satu

#Expected Credit Loss LAF (ECL) (Facility Currency) 
#Expected Credit Loss LAF (ECL) (MYR)
#Expected Credit Loss C&C (ECL) (Facility Currency)
#Expected Credit Loss C&C (ECL) (MYR) 

Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\202408\Finance\ECL"
File = "ECL 0824 - MIS"

#control + F + dropdown new to search bar to find and replace 
import pandas as pd
import numpy as np
import xlsxwriter

#warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

#Stage 3
LAF = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='LAF (2)', header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'
CnC = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='C&C (2)', header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'


In [2]:

Year = 202408
LDB_prev = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year-1)+"\\Loan Database as Jul 2024 (python) - Final.xlsx", sheet_name='Export Worksheet', header=1) # skiprows = 2, nrows=18,  usecols= 'A:D'
MRate = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year)+"\\Working\\00. Month End Rate.xlsx",sheet_name="Sheet1", header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'


In [8]:
LAF.columns = LAF.columns.str.replace("\n", "_")
LAF.columns = LAF.columns.str.replace(" ", "_")
LAF.columns = LAF.columns.str.replace(".", "_")

CnC.columns = CnC.columns.str.replace("\n", "_")
CnC.columns = CnC.columns.str.replace(" ", "_")
CnC.columns = CnC.columns.str.replace(".", "_")


In [9]:
LAF.head(1)

Unnamed:_0 Account_No         Borrower_name   Type      Category  \
0         1.0     500626  Akbank T.A.S. Turkey  Trade  Conventional   

  Unnamed:_5  Stage_1_Conventional  Stage_2_Conventional  Unnamed:_8  \
0        USD                   0.0                   0.0         NaN   

   Stage_1_Islamic  Stage_2_Islamic  Unnamed:_11  Stage_1_Conventional_1  \
0              0.0              0.0          NaN                     0.0   

   Stage_2_Conventional_1  Unnamed:_14  Stage_1_Islamic_1  Stage_2_Islamic_1  
0                     0.0          NaN                0.0                0.0

In [10]:
LAF1 = LAF.iloc[np.where(~(LAF.Account_No.isna()))]

LAF1['LAF_ECL_MYR'] = LAF1['Stage_1_Conventional'] + LAF1['Stage_2_Conventional'] + LAF1['Stage_1_Islamic'] + LAF1['Stage_2_Islamic']

LAF1 = LAF1.fillna(0).groupby(['Account_No','Borrower_name','Category','Unnamed:_5'])[['LAF_ECL_MYR']].sum().reset_index()

LAF1['Unnamed:_5'] = LAF1['Unnamed:_5'].astype(str)
LAF1['Unnamed:_5'] = LAF1['Unnamed:_5'].str.strip()


LAF1.rename(columns={'Account_No':'Account'},inplace=True)
LAF1['Account'] = LAF1['Account'].astype(str)

LDB_prev['Finance(SAP) Number'] = LDB_prev['Finance(SAP) Number'].astype(str)

LDB_prev.columns = LDB_prev.columns.str.replace("\n", "")

LAF1_1 = LAF1.merge(LDB_prev[['Finance(SAP) Number','Facility Currency','Status']].drop_duplicates('Finance(SAP) Number',keep='first').rename(columns={'Finance(SAP) Number':'Account'}),on=['Account'],how='left', suffixes=('_x', '')) #,indicator=True

LAF2 = LAF1_1.rename(columns={'Facility Currency':'Month'}).merge(MRate[['Month','Curr']], on='Month', how='left')

LAF2['LAF_ECL_FC'] = LAF2['LAF_ECL_MYR']/LAF2['Curr']


C:\Users\syahidhalid\AppData\Local\Temp\11\ipykernel_25360\3458547720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAF1['LAF_ECL_MYR'] = LAF1['Stage_1_Conventional'] + LAF1['Stage_2_Conventional'] + LAF1['Stage_1_Islamic'] + LAF1['Stage_2_Islamic']


In [13]:
CnC.head(1)

Unnamed:_0 Account_No         Borrower_name   Type      Category  \
0         1.0     500626  Akbank T.A.S. Turkey  Trade  Conventional   

  Unnamed:_5  Stage_1_Conventional  Stage_2_Conventional  Unnamed:_8  \
0        USD                   0.0                   0.0         NaN   

   Stage_1_Islamic  Stage_2_Islamic  Unnamed:_11  Stage_1_Conventional_1  \
0              0.0              0.0          NaN                     0.0   

   Stage_2_Conventional_1  Unnamed:_14  Stage_1_Islamic_1  Stage_2_Islamic_1  
0                       0          NaN                0.0                0.0

In [14]:
CnC1 = CnC.iloc[np.where(~(CnC.Account_No.isna()))]

CnC1['CnC_ECL_MYR'] = CnC1['Stage_1_Conventional'] + CnC1['Stage_2_Conventional'] + CnC1['Stage_1_Islamic'] + CnC1['Stage_2_Islamic']

CnC1 = CnC1.fillna(0).groupby(['Account_No','Borrower_name','Category','Unnamed:_5'])[['CnC_ECL_MYR']].sum().reset_index()

CnC1_1 = CnC1.merge(LDB_prev[['Finance(SAP) Number','Facility Currency','Status']].drop_duplicates('Finance(SAP) Number',keep='first').rename(columns={'Finance(SAP) Number':'Account_No'}),on=['Account_No'],how='left', suffixes=('_x', '')) #,indicator=True

CnC2 = CnC1_1.rename(columns={'Facility Currency':'Month'}).merge(MRate[['Month','Curr']], on='Month', how='left')

CnC2['CnC_ECL_FC'] = CnC2['CnC_ECL_MYR']/CnC2['Curr']

C:\Users\syahidhalid\AppData\Local\Temp\11\ipykernel_25360\4158385471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CnC1['CnC_ECL_MYR'] = CnC1['Stage_1_Conventional'] + CnC1['Stage_2_Conventional'] + CnC1['Stage_1_Islamic'] + CnC1['Stage_2_Islamic']


In [7]:
merge = pd.concat([LAF2,CnC2])

merge.fillna(0, inplace=True)

merge.Borrower_name = merge.Borrower_name.str.strip()
merge.Borrower_name = merge.Borrower_name.str.upper()

merge1 = merge.fillna(0).groupby(['Account_No','Borrower_name','Category','Month'])[['LAF_ECL_FC',
                                                            'LAF_ECL_MYR','CnC_ECL_FC','CnC_ECL_MYR']].sum().reset_index()


In [15]:
print(sum(merge1.LAF_ECL_FC))
print(sum(merge1.LAF_ECL_MYR))
print(sum(merge1.CnC_ECL_FC))
print(sum(merge1.CnC_ECL_MYR))


65002958.7095442
157806376.47343686
14738696.607395876
19349866.927517027


In [16]:
merge1

Account  \
0    500271   
1    500400   
2    500401   
3    500470   
4    500527   
..      ...   
187      BG   
188      BG   
189      BG   
190      BG   
191     CEL   

                                                                        Borrower_name  \
0                                                 MBI DEVELOPMENT D.O.O, BOSNIA&HERZE   
1                                                                THE REPUBLIC OF IRAQ   
2                                                          THE REPUBLIC OF SEYCHELLES   
3                                                       MINISTRY OF FINANCE - LAO PDR   
4                                                          CAMBODIAN TRANSMISSION LTD   
..                                                                                ...   
187                                                    OM MATERIALS (SARAWAK) SDN BHD   
188                                                       PERTAMA FERROALLOYS SDN BHD   
189  SANTOS FINANCE LIMITED/EXPORT FINANCE AND INSURANCE CORPORATION (EFIC) AUSTRALIA   
190                                                                  SMH RAIL SDN BHD   
191                                                             CERGAS ENERGY LIMITED   

         Category Month  LAF_ECL_FC  LAF_ECL_MYR  CnC_ECL_FC  CnC_ECL_MYR  
0    Conventional   EUR        0.00         0.00        0.00         0.00  
1    Conventional   USD    29576.33    127651.46        0.00         0.00  
2    Conventional   USD    29161.57    125861.32        0.00         0.00  
3    Conventional   USD        0.00         0.00        0.00         0.00  
4         Islamic   USD        0.00         0.00        0.00         0.00  
..            ...   ...         ...          ...         ...          ...  
187  Conventional   USD        0.00         0.00   130491.57    563201.63  
188  Conventional   USD        0.00         0.00   178145.84    768877.43  
189  Conventional   USD        0.00         0.00        0.00         0.00  
190  Conventional   USD        0.00         0.00        0.00         0.00  
191  Conventional   USD        0.00         0.00        0.00         0.00  

[192 rows x 8 columns]

In [17]:
#merge1.to_excel(str(Location)+'\\03. Stage 1 & 2 '+str(File[4:9])+' (python).xlsx', sheet_name='Export Worksheet', index = False)
merge1.to_excel(r'C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\202408\\Working\03. Stage 1 & 2 '+str(File[4:9])+' (python).xlsx', sheet_name='Export Worksheet', index = False)


In [9]:
#str(Location)+'\\04. Stage 1 & 2 '+str(File[4:9])+' (python).xlsx'

In [11]:
str(File[4:9])

'0824 '

In [12]:
merge1['Account_No'].value_counts()

Account_No
BG        7
500271    1
500401    1
500470    1
500527    1
         ..
501150    1
501156    1
501157    1
501158    1
CEL       1
Name: count, Length: 186, dtype: int64

In [13]:
CnC2.iloc[np.where(CnC2['Month']=='MYR')]

Account_No                                                Borrower_name  \
9       500629                         Facade Treatment Engineering Sdn Bhd   
16      500693                     Perusahaan Otomobil Nasional Sdn Bhd- VF   
17      500694                     Perusahaan Otomobil Nasional Sdn Bhd- VF   
21      500727                                           Mewah Datu Sdn Bhd   
22      500729                                       SOP Edible Oil Sdn Bhd   
25      500783                                Mewah Oleo Industries Sdn Bhd   
26      500784                        Felda Global Ventures Capital Sdn Bhd   
27      500785                            Welford Manufacturing (M) Sdn Bhd   
28      500786               Syarikat Motosikal dan Enjin Nasional Sdn. Bhd   
29      500790                                           Mewah Oils Sdn Bhd   
30      500794                                  Tabco Food Services Sdn Bhd   
37      500815                               Central Sugar Refinery Sdn Bhd   
41      500851                                 Foresight Industries Sdn Bhd   
43      500855                                    TS Global Network Sdn Bhd   
44      500859                            Welford Manufacturing (M) Sdn Bhd   
45      500860                                     Lantana Services Sdn Bhd   
46      500861                                    TS Global Network Sdn Bhd   
47      500865                                    Padiberas Nasional Berhad   
49      500878                                         Wok Delights Sdn Bhd   
57      500912                    Ingress Katayama Technical Centre Sdn Bhd   
59      500916                    Ingress Katayama Technical Centre Sdn Bhd   
64      500929                                            Lipidchem Sdn Bhd   
65      500930                                          Usaha Tegar Sdn Bhd   
66      500931                                            Lipidchem Sdn Bhd   
69      500941                                       PWN Excellence Sdn Bhd   
70      500942                                       PWN Excellence Sdn Bhd   
71      500943                                       PWN Excellence Sdn Bhd   
74      500949                           Rhinoshield Industries (M) Sdn Bhd   
75      500952                               UCM Industrial Corporation Bhd   
78      500956                                  Tabco Food Services Sdn Bhd   
80      500965                                     Joyeria Kohinoor Sdn Bhd   
81      500966                                     Joyeria Kohinoor Sdn Bhd   
86      500985                              SSP Raw Metals Tradings Sdn Bhd   
90      500995                                            Zaid Ibrahim & Co   
102     501027                                     Joyeria Kohinoor Sdn Bhd   
103     501030                              SSP Raw Metals Tradings Sdn Bhd   
105     501032                         Tabco Food Services Sdn Bhd (Loan 1)   
106     501033                         Tabco Food Services Sdn Bhd (Loan 2)   
107     501035                                            Bio Eneco Sdn Bhd   
111     501050                               Ingress Industrial (M) Sdn Bhd   
113     501055                        Tabco Food Services Sdn  Bhd (TS0005)   
114     501056                   Taiace Energy Services Sdn Bhd (Tranche 1)   
115     501057                   Taiace Energy Services Sdn Bhd (Tranche 2)   
116     501058                   Taiace Energy Services Sdn Bhd (Tranche 3)   
117     501060                             Joyeria Kohinoor Sdn Bhd - TF-i1   
118     501061                             Joyeria Kohinoor Sdn Bhd - TF-i2   
119     501065                         U.C.M. Industrial Corporation Berhad   
122     501071                  Taiace Energy Services Sdn Bhd (Tranche 4)    
130     501080                              Sri Dayaa Manufacturing Sdn Bhd   
133     501090                Sri Dayaa Manufacturing 

In [24]:
sum(merge.LAF_ECL_MYR)

189277832.60950887

In [20]:
#LAF1.shape

In [26]:
LAF2.head(1)

Account_No       Type      Category Month  LAF_ECL_MYR  Curr  LAF_ECL_FC
0     500271  Non Trade  Conventional   EUR          0.0   5.1         0.0

In [27]:
CnC2.head(1)

Account_No       Type      Category Month  CnC_ECL_MYR  Curr  CnC_ECL_FC
0     500271  Non Trade  Conventional   EUR          0.0   5.1         0.0

In [32]:
LAF

Unnamed:_0 Account_No               Borrower_name       Type  \
0    3308-02230-205-0051-00     500626        Akbank T.A.S. Turkey      Trade   
1    3308-02137-102-0172-00     500891       Amcorp Properties Bhd  Non Trade   
2    3308-04137-102-0214-00     500953       Amcorp Properties Bhd  Non Trade   
3    3308-02224-101-0215-00     500955  Aquawalk (Thailand) Co.Ltd  Non Trade   
4    3308-02205-201-0129-00     500805       Armada Kraken Pte Ltd  Non Trade   
..                      ...        ...                         ...        ...   
168                     NaN        NaN                         NaN        NaN   
169                     NaN        NaN                         NaN        NaN   
170                     NaN        NaN                 GRAND TOTAL        NaN   
171                     NaN        NaN                  As per SAP        NaN   
172                     NaN        NaN                 Differences        NaN   

         Category Unnamed:_5  Stage_1_Conventional  Stage_2_Conventional  \
0    Conventional        USD              0.00e+00              0.00e+00   
1         Islamic        USD              0.00e+00              0.00e+00   
2         Islamic        EUR              0.00e+00              0.00e+00   
3         Islamic        USD              0.00e+00              0.00e+00   
4    Conventional        USD              0.00e+00              0.00e+00   
..            ...        ...                   ...                   ...   
168           NaN        NaN                   NaN                   NaN   
169           NaN        NaN                   NaN                   NaN   
170           NaN        NaN              9.30e+07              5.09e+07   
171           NaN        NaN             -9.30e+07             -5.09e+07   
172           NaN        NaN              1.75e-02             -2.59e-02   

     Unnamed:_8  Stage_1_Islamic  Stage_2_Islamic  Unnamed:_11  \
0      0.00e+00         0.00e+00         0.00e+00     0.00e+00   
1      0.00e+00         0.00e+00         0.00e+00     0.00e+00   
2      0.00e+00         0.00e+00         0.00e+00     0.00e+00   
3      0.00e+00         0.00e+00         0.00e+00     0.00e+00   
4      0.00e+00         0.00e+00         0.00e+00     0.00e+00   
..          ...              ...              ...          ...   
168         NaN              NaN              NaN          NaN   
169         NaN              NaN              NaN          NaN   
170    1.44e+08         3.44e+07         1.09e+07     1.89e+08   
171    1.44e+08        -3.44e+07        -1.09e+07     1.89e+08   
172    0.00e+00         3.21e-02        -1.43e-02          NaN   

     Stage_1_Conventional_1  Stage_2_Conventional_1  Unnamed:_14  \
0                  0.00e+00                0.00e+00          NaN   
1                  0.00e+00                0.00e+00          NaN   
2                  0.00e+00                0.00e+00          NaN   
3                  0.00e+00                0.00e+00          NaN   
4                  0.00e+00                0.00e+00          NaN   
..                      ...                     ...          ...   
168                     NaN                     NaN          NaN   
169                     NaN                     NaN          NaN   
170               -1.36e+06               -4.31e+06          NaN   
171               -1.36e+06               -4.31e+06          0.0   
172                2.97e-03               -8.27e-04          0.0   

     Stage_1_Islamic_1  Stage_2_Islamic_1  
0             0.00e+00           0.00e+00  
1             0.00e+00           0.00e+00  
2             0.00e+00           0.00e+00  
3             0.00e+00           0.00e+00  
4             0.00e+00           0.00e+00  
..                 ...                ...  
168                NaN                NaN  
169                NaN                NaN  
170           1.48e+06           2.95e+06  
171           1.48e+06           2.95e+06  
172          -1.78e-03          -1.75